<a href="https://colab.research.google.com/github/ayuadmy/simpasi/blob/main/SIMPASI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Needed


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q tensorflowjs
!pip install -q scann

In [ ]:
import pathlib
import os
import pprint
import tempfile
from google.colab import files


from typing import Dict, Text

import csv
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflowjs as tfjs
import tensorflow_recommenders as tfrs
print(tf. __version__)

2.8.2


# MPASI Menu, Recipe, Ingredients Dataset

In [ ]:
#Download dataset from google drive
#Bahan_Makanan
!gdown --id 1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS\

#Resep
!gdown --id 1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT\

#Komposisi
!gdown --id 1xufQ7DEIiKIcWWd0sOYXMLkjJQ8kYgvJ\

#Menu Makanan
!gdown --id 1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb\

#Feedback
!gdown --id 1ddU24ajNxXtljNaBjhwY0kx0SvqxqLzC\

#Feedback Bahan
!gdown --id 1GevXM5jVP6jD0wtWPi9LOIxSs3EzHLrY\

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS
To: /content/bahan_makanan.csv
100% 5.64k/5.64k [00:00<00:00, 8.56MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT
To: /content/resep.csv
100% 3.57k/3.57k [00:00<00:00, 5.32MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From:

In [ ]:
#Move files to tmp (To be deleted after closing this google colab in some time)
!mv "/content/bahan_makanan.csv" "/tmp"
!mv "/content/resep.csv" "/tmp"
!mv "/content/komposisi_baru.csv" "/tmp"
!mv "/content/kombinasi_menu.csv" "/tmp"
!mv "/content/feedback_fix.csv" "/tmp"
!mv "/content/Feedback_Bahan.csv" "/tmp"

In [ ]:
#Create directory of datasets
bahan_makanan_dir = "/tmp/bahan_makanan.csv"
resep_dir = "/tmp/resep.csv"
komposisi_dir = "/tmp/komposisi_baru.csv"
kombinasi_menu_dir = "/tmp/kombinasi_menu.csv"
feedback_dir = "/tmp/feedback_fix.csv"
feedback_bahan_dir = "/tmp/Feedback_Bahan.csv"

In [ ]:
bahan_makanan = pd.read_csv(bahan_makanan_dir)
bahan_makanan.head()

,id_bahan,nama_bahan
0,0,Agar-Agar
1,1,Agar Laut
2,2,Akar Tonjong
3,3,Alpukat
4,4,Ampas Tahu


In [ ]:
dict_komposisi = {}

with open(komposisi_dir, mode="r") as data_komposisi:
  reader = csv.reader(data_komposisi)
  keys = next(reader)
  for row in reader:
    if row[0] not in dict_komposisi.keys():
      dict_komposisi[row[0]] = row[2]
    else:
      dict_komposisi[row[0]] = dict_komposisi[row[0]] + ", " + row[2]
    
data = {keys[0]: dict_komposisi.keys(), keys[2]: dict_komposisi.values()}
komposisi = pd.DataFrame.from_dict(data)
komposisi.head()

,id_resep,nama_bahan
0,0,"Blewah, Melon, Agar-Agar"
1,1,Apel
2,2,"Apel, Margarine, Oatmeal"
3,3,"Tahu Sutra, Pisang, Apel"
4,4,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."


In [ ]:
resep = pd.read_csv(resep_dir)
resep.head()

,id_resep,nama_resep
0,0,Agar-agar Isi Blewah
1,1,Apel Rebus
2,2,Apple Crumble
3,3,Apricot Tahu
4,4,Ayam Zukini Hati


In [ ]:
komposisi['id_resep'] = komposisi['id_resep'].astype(int)
komposisi_resep = komposisi.merge(resep, on=['id_resep', 'id_resep'], how='left')
#komposisi_resep

del(komposisi_resep['id_resep'])
komposisi_resep = komposisi_resep[['nama_resep', 'nama_bahan']]
komposisi_resep

,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
2,Apple Crumble,"Apel, Margarine, Oatmeal"
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
4,Ayam Zukini Hati,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."
...,...,...
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."
156,Tumis Ikan Dori,"Bawang Merah, Wortel, Ketimun, Bawang Putih, T..."
157,Tuna Mornay,"Saus Pasta, Ikan Tuna, Roti Tawar, Keju, Krim ..."
158,Yoghurt Buah Pop,"Strawbery, Yoghurt"


In [ ]:
menu_harian = pd.read_csv(kombinasi_menu_dir)

menu_harian["malam"].fillna("", inplace = True) 
menu_harian["snack1"].fillna("", inplace = True) 
menu_harian["snack2"].fillna("", inplace = True) 

menu_harian.head()

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid
0,1,Apel Rebus,Apel Rebus,,,,1,0,0,0,144.00,38.12,0.72,0.46
1,2,Bubur Apel,Bubur Apel,,,,1,0,0,0,144.50,34.52,0.66,0.42
2,3,Bubur Kacang Polong,Bubur Kacang Polong,,,,1,0,0,0,127.44,15.60,3.84,5.52
3,4,Bubur Kembang Kol,Bubur Kembang Kol,,,,1,0,0,0,143.20,21.12,9.46,2.32
4,5,Bubur Plum,Bubur Plum,,,,1,0,0,0,128.10,30.88,1.10,0.02


In [ ]:
menu_harian.to_csv('menu_harian.csv')
files.download("menu_harian.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
list_resep = [a for a in komposisi_resep["nama_resep"]]

cek_resep = menu_harian.drop(["id_kombinasi","bulan_6","bulan_7_8","bulan_9_11","bulan_12_24","total_energi","total_carbo","total_protein","total_lipid"], axis = 1)
komposisi_resep_filtered = komposisi_resep.copy()

for a in list_resep:
  mask = cek_resep.pagi.isin([a])
  if mask.any():
    print("ada bahan pagi {}".format(a))
  else:
    mask = cek_resep.siang.isin([a])
    if mask.any():
      print("ada bahan siang {}".format(a))
    else:
      mask = cek_resep.malam.isin([a])
      if mask.any():
        print("ada bahan malam {}".format(a))
      else:
        mask = cek_resep.snack1.isin([a])
        if mask.any():
          print("ada bahan snack1 {}".format(a))
        else:
          mask = cek_resep.snack2.isin([a])
          if mask.any():
            print("ada bahan snack2 {}".format(a))
          else:
            print("tidak ada bahan {}".format(a))
            komposisi_resep_filtered.drop(komposisi_resep_filtered.index[komposisi_resep_filtered["nama_resep"] == a], inplace=True)

komposisi_resep_filtered

ada bahan snack1 Agar-agar Isi Blewah
ada bahan pagi Apel Rebus
tidak ada bahan Apple Crumble
ada bahan siang Apricot Tahu
tidak ada bahan Ayam Zukini Hati
tidak ada bahan Bayam Panggang
tidak ada bahan Biskuit Beras Merah
ada bahan snack1 Biskuit Oat Ceri
ada bahan snack1 Biskuit Oat Wortel
ada bahan snack1 Biskuit Sereal
ada bahan snack1 Bola-bola Ubi
tidak ada bahan Bola Ikan
tidak ada bahan Brokoli Keju
tidak ada bahan Brokoli Rebus
tidak ada bahan Buah Kebab Dingin
tidak ada bahan Bubur Alpukat
ada bahan pagi Bubur Apel
tidak ada bahan Bubur Apel Bayam Hijau
tidak ada bahan Bubur Apel Bayam Merah
tidak ada bahan Bubur Apel Pisang
tidak ada bahan Bubur Apricot
tidak ada bahan Bubur Aprikot Yogurt
ada bahan pagi Bubur Ayam Kedelai
ada bahan siang Bubur Ayam Talas Keju
tidak ada bahan Bubur Beras
tidak ada bahan Bubur Beras Brocang
ada bahan siang Bubur Beras Merah
tidak ada bahan Bubur Beras Poldu
tidak ada bahan Bubur Bit Ceri
ada bahan pagi Bubur Buah dan Sayur
ada bahan pagi Bubu

,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
7,Biskuit Oat Ceri,"Unsalted Butter, Oatmeal, Ceri, Maizena"
8,Biskuit Oat Wortel,"Maizena, Unsalted Butter, Wortel, Oatmeal"
...,...,...
147,Sup Labu Kuning,Labu Kuning
149,Sup Mihun Ikan Gabus,"Bihun, Gabus, Kaldu Ikan, Bawang Putih, Margar..."
154,Tim Lele,"Tahu Sutra, Wortel, Belut, Beras"
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."


In [ ]:
komposisi_resep_filtered.to_csv('komposisi_resep_filtered.csv')
files.download("komposisi_resep_filtered.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = list(list_set)
    unique_list.sort()

    return unique_list

In [ ]:
resep_bulan_6 = []
resep_bulan_7_8 = []
resep_bulan_9_11 = []
resep_bulan_12_24 = []

cek_bulan = menu_harian.drop(["id_kombinasi","total_energi","total_carbo","total_protein","total_lipid"], axis = 1)

#bulan 6
mask = menu_harian.bulan_6.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_6.append(cek_bulan.pagi[x])
    resep_bulan_6.append(cek_bulan.siang[x])

resep_bulan_6 = unique(resep_bulan_6)
print(len(resep_bulan_6))
print(resep_bulan_6)

#bulan 7 8
mask = menu_harian.bulan_7_8.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_7_8.append(cek_bulan.pagi[x])
    resep_bulan_7_8.append(cek_bulan.siang[x])
    resep_bulan_7_8.append(cek_bulan.malam[x])

resep_bulan_7_8 = unique(resep_bulan_7_8)
print(len(resep_bulan_7_8))
print(resep_bulan_7_8)

#bulan 9 11
mask = menu_harian.bulan_9_11.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_9_11.append(cek_bulan.pagi[x])
    resep_bulan_9_11.append(cek_bulan.siang[x])
    resep_bulan_7_8.append(cek_bulan.malam[x])
    resep_bulan_7_8.append(cek_bulan.snack1[x])

resep_bulan_9_11 = unique(resep_bulan_9_11)
print(len(resep_bulan_9_11))
print(resep_bulan_9_11)

#bulan 12 24
mask = menu_harian.bulan_12_24.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_12_24.append(cek_bulan.pagi[x])
    resep_bulan_12_24.append(cek_bulan.siang[x])
    resep_bulan_12_24.append(cek_bulan.malam[x])
    resep_bulan_12_24.append(cek_bulan.snack1[x])
    resep_bulan_12_24.append(cek_bulan.snack2[x])

resep_bulan_12_24 = unique(resep_bulan_12_24)
print(len(resep_bulan_12_24))
print(resep_bulan_12_24)


16
['Apel Rebus', 'Bubur Apel', 'Bubur Kacang Polong', 'Bubur Kembang Kol', 'Bubur Plum', 'Bubur Singkong Apel', 'Bubur Susu Kacang Polong', 'Bubur Ubi Kembang Kol', 'Pure Alpukat dan Pepaya', 'Pure Apel dan Papaya', 'Pure Jagung Manis', 'Pure Pisang dan Pepaya', 'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning', 'Pure Tiga Rasa', 'Pure Ubi Kuning', 'Pure Ubi Kuning dan Beras Merah']
19
['Bubur Buah dan Sayur', 'Bubur Ceri Alpukat', 'Bubur Jagung Bayam Tahu', 'Bubur Kentang', 'Bubur Kentang Edamame', 'Bubur Labu Brokoli', 'Bubur Oatmeal', 'Bubur Pepaya', 'Bubur Pir', 'Bubur Pisang', 'Bubur Pisang Jeruk', 'Bubur Pisang Ubi', 'Bubur Talas Apel', 'Bubur Tempe Beras Merah', 'Bubur Tomat Kacang Merah', 'Bubur Ubi Bayam Kacang Polong', 'Bubur Wortel', 'Pisang Mangga Rebus', 'Sayur Kukus']
20
['Bubur Ayam Kedelai', 'Bubur Ayam Talas Keju', 'Bubur Ceri Alpukat', 'Bubur Jagung Bayam Tahu', 'Bubur Jagung Wortel', 'Bubur Kacang Hijau Kurma', 'Bubur Kentang', 'Bubur Kentang Edamame', 'Bubur Labu B

# Simulasi Aplikasi SIMPASI - User Baru

Pada user baru, customer akan melewati tahap-tahap berikut akan berupa:

1.   Log-in/Sign-up dengan email dan passsword
2.   Meminta data bayi berupa nama, tanggal lahir, bahan yang pernah dicoba sebelumnya (suka/tidak suka/alergi)
3.   Mendapatkan rekomendasi makanan dari riwayat makanan yang disukai/tidak disukai/alergi




Sehingga, sistem rekomendasi harus melakukan:


1.   Meminta username/email/password untuk masuk ke akun
2.   Meminta nama, tanggal lahir, bahan yang pernah dicoba sebelumnya *(suka/tidak suka/alergi)
3.   Menawarkan resep berdasarkan data bahan
4.   Menghitung Umur dan EER anak
5.   Merekomendasikan resep yang sesuai dengan profil
6.   Memfilter resep berdasarkan alergi bahan
7.   Memfilter menu makanan berdasarkan umur dan EER
8.   Menampilkan kombinasi menu dari resep yang cocok



## I. *Contoh Profil User*

In [ ]:
#Contoh
# email = []
email = str(input("Masukkan email" + ": "))
# email.append(x)

# username = []
username = str(input("Masukkan Nama Username" + ": "))
# username.append(x)

# password = []
password = str(input("Masukkan Password" + ": "))
# password.append(x)

# nama_bayi = []
nama_bayi = str(input("Masukkan nama" + ": "))
# nama_bayi.append(x)

# tanggal_lahir = []
tanggal_lahir = str(input("Masukkan tanggal lahir" + ": "))
# tanggal_lahir.append(x)
#"17/07/2021"

id_bayi = 101
# id_bayi.append(int(len(tanggal_lahir)+100))
#"101"

# berat_badan = []
berat_badan = float(input("Masukkan Berat Badan bayi" + ": "))
# berat_badan.append(x)
#6.9

print(email)
print(username)
print(nama_bayi)
print(tanggal_lahir)
print(id_bayi)
print(berat_badan)

Masukkan email: fatursrahman@gmail.com
Masukkan Nama Username: Fathur
Masukkan Password: qwe
Masukkan nama: tor
Masukkan tanggal lahir: 17/07/2021
Masukkan Berat Badan bayi: 6.9
fatursrahman@gmail.com
Fathur
tor
17/07/2021
101
6.9


## II. Cek 8 Bahan (Suka, Tidak Suka, Alergi, Tidak tahu) dan Feedback Awal 3 Resep


In [ ]:
#Menampilkan Nama Bahan Berdasarkan ID
def ingredient_name(id):
  return bahan_makanan.loc[id, 'nama_bahan']

#driver code
print(ingredient_name(1))

Agar Laut


In [ ]:
#Bahan Alergi Umum: Susu, Telur Ayam, Telur Bebek, Kacang Tanah, Kacang Kedelai, Udang, Kerang, Cumi-Cumi, Tepung Beras, Roti, Daging Sapi, Pisang, Alpukat
tes_bahan = ["Susu", "Telur Ayam", "Telur Bebek", "Kacang Tanah", "Kacang Kedelai", "Udang", "Kerang", "Cumi-Cumi", "Tepung Beras", "Roti Tawar", "Daging Sapi", "Pisang", "Alpukat"]
jumlah_tes = 8
alergi = []
tidak_suka = []
suka = []
tidak_tahu = []

import random
tes_bahan_random = random.sample(tes_bahan, jumlah_tes)

#Meminta 8 Feedback Bahan dari User
print("Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan")
print("Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu")
for i in range(len(tes_bahan_random)):

  x = int(input(tes_bahan_random[i] + ": "))

  if x == 2:
    tidak_suka.append(tes_bahan_random[i])
  elif x == 3:
    alergi.append(tes_bahan_random[i])
  elif x == 4:
    tidak_tahu.append(tes_bahan_random[i])
  elif x == 1:
    suka.append(tes_bahan_random[i])

print(tidak_suka)
print(suka)
print(tidak_tahu)
print(alergi)

Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan
Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu
Daging Sapi: 1
Cumi-Cumi: 1
Telur Bebek: 4
Pisang: 4
Telur Ayam: 4
Kerang: 3
Susu: 4
Udang: 4
[]
['Daging Sapi', 'Cumi-Cumi']
['Telur Bebek', 'Pisang', 'Telur Ayam', 'Susu', 'Udang']
['Kerang']


## III. *Hitung Umur dan EER*

In [ ]:
from datetime import datetime
from datetime import date

#Fungsi Menghitung Umur User
def ageCalculator(birthdate):
  my_birthdate = datetime.strptime(birthdate, "%d/%m/%Y")
  my_year = my_birthdate.year
  my_month = my_birthdate.month
  my_day = my_birthdate.year
  
  born = my_birthdate.date()
  today = date.today()
  
  years = ((today-born).total_seconds()/ (365.242*24*3600))
  yearsInt = int(years)
  months = (years-yearsInt)*12
  monthsInt = int(months)
  days = (months-monthsInt)*(365.242/12)
  daysInt = int(days)

  return yearsInt, monthsInt, daysInt
  

#Fungsi Menghitung EER (Estimated Energy Requirements)
def eerCalculator(year, month, day, weight):
  if month == 6:
    EER = (89*weight - 100) + 56
  elif 7 <= month <= 12:
    EER = (89*weight - 100) + 22
  elif 13 <= month <= 35:
    EER = (89*weight - 100) + 20
  
  return EER
  
#Driver Code
#Cari Umur User (Tahun, Bulan, Tanggal)
tahun, bulan, hari = ageCalculator(tanggal_lahir)
print("Sekarang kamu berumur {0} tahun, {1} bulan, {2} hari.".format(tahun , bulan, hari))

#Cari EER User (+-10%)
eer = eerCalculator(tahun, bulan, hari, berat_badan)
print("Kebutuhan energi harianmu: {0} - {1} Joule".format(int(eer*0.9), int(eer*1.1)))

Sekarang kamu berumur 0 tahun, 10 bulan, 20 hari.
Kebutuhan energi harianmu: 482 - 589 Joule


## IV. *Model Sistem Rekomendasi Menu MPASI*

In [ ]:
df = pd.read_csv(feedback_dir)
feedback_slices = tf.data.Dataset.from_tensor_slices(dict(df))

In [ ]:
feedback_slices.element_spec

{'hari': TensorSpec(shape=(), dtype=tf.string, name=None),
 'id_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'id_resep': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'jumlah_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'keterangan': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nilai_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [ ]:
feedback_slices

<TensorSliceDataset element_spec={'id_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id_resep': TensorSpec(shape=(), dtype=tf.int64, name=None), 'nilai_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'hari': TensorSpec(shape=(), dtype=tf.string, name=None), 'keterangan': TensorSpec(shape=(), dtype=tf.string, name=None), 'jumlah_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
komposisi_resep_filtered_slices = tf.data.Dataset.from_tensor_slices(dict(komposisi_resep_filtered))

In [ ]:
komposisi_resep_filtered_slices.element_spec

{'nama_bahan': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
feedback = feedback_slices.map(lambda x: {
    "nama_resep": x["nama_resep"],
    "user_id": tf.strings.as_string(x["user_id"]),
})
menu = komposisi_resep_filtered_slices.map(lambda x: x["nama_resep"])

In [ ]:
print(feedback)

In [ ]:
tf.random.set_seed(12)
shuffled = feedback.shuffle(100, seed=12, reshuffle_each_iteration=False)

train = shuffled.take(80)
test = shuffled.skip(80).take(20)

In [ ]:
nama_resep = menu.batch(10)
user_ids = feedback.batch(100).map(lambda x: (x["user_id"]))

unique_menu = np.unique(np.concatenate(list(nama_resep)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
print(nama_resep.element_spec)
print(user_ids.element_spec)

TensorSpec(shape=(None,), dtype=tf.string, name=None)
TensorSpec(shape=(None,), dtype=tf.string, name=None)


In [ ]:
print(len(unique_menu))
print(unique_menu)
print(len(unique_user_ids))
print(unique_user_ids)

77
[b'Agar-agar Isi Blewah' b'Apel Rebus' b'Apricot Tahu' b'Biskuit Oat Ceri'
 b'Biskuit Oat Wortel' b'Biskuit Sereal' b'Bola-bola Ubi' b'Bubur Apel'
 b'Bubur Ayam Kedelai' b'Bubur Ayam Talas Keju' b'Bubur Beras Merah'
 b'Bubur Buah dan Sayur' b'Bubur Ceri Alpukat' b'Bubur Hunkwe Susu Keju'
 b'Bubur Jagung Bayam Tahu' b'Bubur Jagung Susu' b'Bubur Jagung Wortel'
 b'Bubur Kacang Hijau Kurma' b'Bubur Kacang Polong' b'Bubur Kembang Kol'
 b'Bubur Kentang' b'Bubur Kentang Edamame' b'Bubur Labu Brokoli'
 b'Bubur Makaroni Teri Bayam' b'Bubur Oatmeal' b'Bubur Pepaya'
 b'Bubur Pir' b'Bubur Pisang' b'Bubur Pisang Jeruk'
 b'Bubur Pisang Kentang' b'Bubur Pisang Tomat' b'Bubur Pisang Ubi'
 b'Bubur Plum' b'Bubur Sawi Wortel' b'Bubur Singkong Apel'
 b'Bubur Susu Kacang Polong' b'Bubur Tahu' b'Bubur Tahu Wortel'
 b'Bubur Talas Apel' b'Bubur Tempe' b'Bubur Tempe Beras Merah'
 b'Bubur Tomat Kacang Merah' b'Bubur Ubi Bayam Kacang Polong'
 b'Bubur Ubi Kembang Kol' b'Bubur Wortel' b'Daging Ungkep'
 b'Ikan K

In [ ]:
embedding_dimension = 16

In [ ]:
#Query Tower
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
#Candidate Tower
menu_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_menu, mask_token=None),
  tf.keras.layers.Embedding(len(unique_menu) + 1, embedding_dimension)
])

In [ ]:
#metrics
metrics = tfrs.metrics.FactorizedTopK(
  candidates=menu.batch(64).map(menu_model)
)

In [ ]:
#loss
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class SIMPASIModel(tfrs.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_menu_embeddings = self.menu_model(features["nama_resep"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_menu_embeddings)

In [ ]:
class NoBaseClassSIMPASIModel(tf.keras.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_menu_embeddings = self.menu_model(features["nama_resep"])
      loss = self.task(user_embeddings, positive_menu_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_menu_embeddings = self.movie_model(features["nama_resep"])
    loss = self.task(user_embeddings, positive_menu_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [ ]:
model = SIMPASIModel(user_model, menu_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(10).batch(16).cache()
cached_test = test.batch(8).cache()

In [ ]:
cached_train.element_spec

{'nama_resep': TensorSpec(shape=(None,), dtype=tf.string, name=None),
 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None)}

In [ ]:
print(cached_test)

<CacheDataset element_spec={'nama_resep': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [ ]:
history = model.fit(cached_train, epochs=10)

Epoch 1/10
5/5 [==============================] - 2s 50ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0625 - factorized_top_k/top_10_categorical_accuracy: 0.1625 - factorized_top_k/top_50_categorical_accuracy: 0.7250 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.3568 - regularization_loss: 0.0000e+00 - total_loss: 44.3568
Epoch 2/10
5/5 [==============================] - 0s 53ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0625 - factorized_top_k/top_5_categorical_accuracy: 0.4750 - factorized_top_k/top_10_categorical_accuracy: 0.6875 - factorized_top_k/top_50_categorical_accuracy: 0.9750 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.2037 - regularization_loss: 0.0000e+00 - total_loss: 44.2037
Epoch 3/10
5/5 [==============================] - 0s 46ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0750 - factorized_top_k/top_5_categorical_accuracy: 0.8375 - facto

In [ ]:
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 0s 53ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.1500 - factorized_top_k/top_10_categorical_accuracy: 0.2000 - factorized_top_k/top_50_categorical_accuracy: 0.6500 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 11.4629 - regularization_loss: 0.0000e+00 - total_loss: 11.4629


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.20000000298023224,
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.6499999761581421,
 'factorized_top_k/top_5_categorical_accuracy': 0.15000000596046448,
 'loss': 6.065029144287109,
 'regularization_loss': 0,
 'total_loss': 6.065029144287109}

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((menu.batch(100), menu.batch(100).map(model.menu_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["10"]))
print(f"Recommendations for user 10: {titles}")

Recommendations for user 10: [[b'Bubur Kentang Edamame' b'Pure Ubi Kuning dan Beras Merah'
  b'Bubur Susu Kacang Polong' b'Bubur Singkong Apel' b'Pure Tiga Rasa'
  b'Bubur Oatmeal' b'Perkedel Lele' b'Bubur Wortel' b'Daging Ungkep'
  b'Bola-bola Ubi']]


In [ ]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:

  # Save the index.
  tf.saved_model.save(obj=index, export_dir="export")

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load("export")

  print(loaded)
  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["32"])

  print(f"Recommendations: {titles[0][:3]}")
  loaded

INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7ff0db0fa790>
Recommendations: [b'Bubur Talas Apel' b'Bubur Jagung Wortel' b'Kukis Havermut Apel']


In [ ]:
#tflite
converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

12392

In [ ]:
#h5
model.save_weights('model_weight', save_format='tf')

In [ ]:
pb_model_dir = "/content/saved_model.pb"
h5_model = "/mymodel.h5"

In [ ]:
# Loading the Tensorflow Saved Model (PB)
model_v2 = tf.keras.models.load_model(pb_model_dir)
print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model_v2, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())

OSError: ignored

In [ ]:
model = tf.keras.models.load_model(pb_model_dir)
print(model.summary())

NameError: ignored

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ValueError: ignored

In [ ]:
tflite_model_file = pathlib.Path("/content/converted_model.tflite")
tflite_model_file.write_bytes(tflite_model)

12392

In [ ]:
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Tensorflow JS
# import time
# saved_model_path = "tmp/saved_models/{}".format(int(time.time()))

# tf.keras.experimental.export_saved_model(model,saved_model_path)

model.save("/content/")

ValueError: ignored

In [ ]:
# tensorflowjs_converter --input_format=keras /content/model_weight /tmp/tfjs_model
tensorflowjs_converter \
    --input_format=tf_saved_model \
    /content/model_weight \
    /tmp/tfjs_model
# tfjs.converters.save_keras_model(model_weight,"model_weights")

In [ ]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

print(input_details[0])
# Test the model.
interpreter.set_tensor(input_details[0]["index"], np.array(["32"]))

interpreter.invoke()

reccomendation = interpreter.get_tensor(output_details[1]['index'])
rating = interpreter.get_tensor(output_details[0]['index'])
print(reccomendation)
print(rating)

# Print input shape and type
print('{} input(s):'.format(len(input_details)))
for i in range(0, len(input_details)):
    print('{} {}'.format(input_details[i]['shape'], input_details[i]['dtype']))

# Print output shape and type
print('\n{} output(s):'.format(len(output_details)))
for i in range(0, len(output_details)):
    print('{} {}'.format(output_details[i]['shape'], output_details[i]['dtype']))

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 10, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall_1:1', 'index': 12, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtyp

In [ ]:
np.array(["32"])

array(['32'], dtype='<U2')

In [ ]:
print(type(titles))            # <type 'EagerTensor'>
print(type(titles.numpy()))    # <type 'numpy.ndarray'>
print(type(np.array(titles)))  # <type 'numpy.ndarray'>

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
titles_numpy = np.array(titles).transpose().astype(str)

rekomendasi = pd.DataFrame(titles_numpy, columns=["rekomendasi_resep"])
rekomendasi

,rekomendasi_resep
0,Bubur Kentang Edamame
1,Pure Ubi Kuning dan Beras Merah
2,Bubur Susu Kacang Polong
3,Bubur Singkong Apel
4,Pure Tiga Rasa
5,Bubur Oatmeal
6,Perkedel Lele
7,Bubur Wortel
8,Daging Ungkep
9,Bola-bola Ubi


## V. *Filtering Rekomendasi Resep Berdasarkan Alergi*

In [ ]:
def search_list(list1):
  pattern = '|'.join([f'(?i){list_value}' for list_value in list1])
  return pattern

In [ ]:
list_rekomendasi = [a for a in rekomendasi["rekomendasi_resep"]]
rekomendasi_nonalergen = rekomendasi.copy()

for x in list_rekomendasi:
  print(x)
  index_x = komposisi_resep_filtered.index[komposisi_resep_filtered["nama_resep"] == x].tolist()
  print(index_x)
  result = komposisi_resep_filtered.loc[index_x, ["nama_bahan"]]
  mask = result.nama_bahan.str.contains(search_list(alergi))
  print(mask)
  if mask.any():
    rekomendasi_nonalergen = rekomendasi_nonalergen[rekomendasi_nonalergen["rekomendasi_resep"] != x]
    
print(len(rekomendasi_nonalergen))
rekomendasi_nonalergen

Bubur Kentang Edamame
[43]
43    False
Name: nama_bahan, dtype: bool
Pure Ubi Kuning dan Beras Merah
[134]
134    False
Name: nama_bahan, dtype: bool
Bubur Susu Kacang Polong
[65]
65    False
Name: nama_bahan, dtype: bool
Bubur Singkong Apel
[63]
63    False
Name: nama_bahan, dtype: bool
Pure Tiga Rasa
[132]
132    False
Name: nama_bahan, dtype: bool
Bubur Oatmeal
[50]
50    False
Name: nama_bahan, dtype: bool
Perkedel Lele
[121]
121    False
Name: nama_bahan, dtype: bool
Bubur Wortel
[78]
78    False
Name: nama_bahan, dtype: bool
Daging Ungkep
[82]
82    False
Name: nama_bahan, dtype: bool
Bola-bola Ubi
[10]
10    False
Name: nama_bahan, dtype: bool
Bubur Ubi Kembang Kol
[77]
77    False
Name: nama_bahan, dtype: bool
Bubur Pisang Ubi
[59]
59    False
Name: nama_bahan, dtype: bool
Bubur Pisang Tomat
[58]
58    False
Name: nama_bahan, dtype: bool
Pisang Mangga Rebus
[123]
123    False
Name: nama_bahan, dtype: bool
Jus Pisang Kurma
[92]
92    False
Name: nama_bahan, dtype: bool
Bubur Ape

,rekomendasi_resep
0,Bubur Kentang Edamame
1,Pure Ubi Kuning dan Beras Merah
2,Bubur Susu Kacang Polong
3,Bubur Singkong Apel
4,Pure Tiga Rasa
5,Bubur Oatmeal
6,Perkedel Lele
7,Bubur Wortel
8,Daging Ungkep
9,Bola-bola Ubi


## VI. *Filtering Rekomendasi Resep Berdasarkan umur dan EER*

In [ ]:
def filter_umur(rsp,rspbln):
  list_rsp = [a for a in rsp["rekomendasi_resep"]]
  output = rsp.copy()

  for x in list_rsp:
    if x in rspbln:
      print(x + " Exists")
    else:
      output = output[output["rekomendasi_resep"] != x]
  return output

In [ ]:
if bulan <= 6:
  rekomendasi_nonalergen_umur = filter_umur(rekomendasi_nonalergen, resep_bulan_6)
elif bulan in (7, 8):
  rekomendasi_nonalergen_umur = filter_umur(rekomendasi_nonalergen, resep_bulan_7_8)
elif bulan in (9, 10, 11):
  rekomendasi_nonalergen_umur = filter_umur(rekomendasi_nonalergen, resep_bulan_9_11)
else:
  rekomendasi_nonalergen_umur = filter_umur(rekomendasi_nonalergen, resep_bulan_12_24)

print(rekomendasi_nonalergen_umur)

Bubur Kentang Edamame Exists
Bubur Wortel Exists
Bubur Pisang Ubi Exists
Bubur Labu Brokoli Exists
Bubur Kacang Hijau Kurma Exists
Bubur Pisang Kentang Exists
Tim Lele Exists
Bubur Tahu Wortel Exists
Bubur Pepaya Exists
           rekomendasi_resep
0      Bubur Kentang Edamame
7               Bubur Wortel
11          Bubur Pisang Ubi
21        Bubur Labu Brokoli
23  Bubur Kacang Hijau Kurma
24      Bubur Pisang Kentang
31                  Tim Lele
39         Bubur Tahu Wortel
40              Bubur Pepaya


In [ ]:
def filter_menu_harian(rekomend, menu, mon):
  menu_copy = menu.copy()
  menu_copy['menu_makanan'] = menu_copy["pagi"] + ',' + menu_copy["siang"] + ',' + menu_copy["malam"] + ',' + menu_copy["snack1"] + ',' + menu_copy["snack2"]
  list_rekomend = [a for a in rekomend["rekomendasi_resep"]]
  
  if mon <= 6:
    menu_copy = menu_copy.iloc[0:15]
  elif mon in (7, 8):
    menu_copy = menu_copy.iloc[16:783]
  elif mon in (9, 10, 11):
    menu_copy = menu_copy.iloc[784:20239]
  else:
    menu_copy = menu_copy.iloc[20240:20383]
  
  mask = menu_copy.menu_makanan.str.contains(search_list(list_rekomend))
  menu_copy = menu_copy[mask]
  return menu_copy

filter_menu_harian(rekomendasi_nonalergen_umur, menu_harian, 11)

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid,menu_makanan
841,842,Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur Kacang Hijau Kurma,Agar-agar Isi Blewah,,0,0,1,0,346.100,44.100,19.030,10.4975,"Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur..."
842,843,Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur Kacang Hijau Kurma,Biskuit Oat Ceri,,0,0,1,0,346.200,38.730,18.593,12.8110,"Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur..."
843,844,Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur Kacang Hijau Kurma,Biskuit Oat Wortel,,0,0,1,0,348.650,38.800,18.660,13.0100,"Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur..."
844,845,Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur Kacang Hijau Kurma,Biskuit Sereal,,0,0,1,0,347.300,40.780,18.970,11.8500,"Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur..."
845,846,Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur Kacang Hijau Kurma,Bola-bola Ubi,,0,0,1,0,346.040,40.860,19.160,10.5360,"Bubur Ayam Kedelai,Bubur Ayam Talas Keju,Bubur..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20234,20235,Sayur Kukus,Tim Lele,Sayur Kukus,Kukis Oatmeal Keju,,0,0,1,0,345.960,59.030,17.180,4.5700,"Sayur Kukus,Tim Lele,Sayur Kukus,Kukis Oatmeal..."
20235,20236,Sayur Kukus,Tim Lele,Sayur Kukus,Lapis Mangga,,0,0,1,0,347.545,63.930,16.700,2.6370,"Sayur Kukus,Tim Lele,Sayur Kukus,Lapis Mangga,"
20236,20237,Sayur Kukus,Tim Lele,Sayur Kukus,Pancake Labu,,0,0,1,0,346.760,60.850,17.660,3.8300,"Sayur Kukus,Tim Lele,Sayur Kukus,Pancake Labu,"
20237,20238,Sayur Kukus,Tim Lele,Sayur Kukus,Perkedel Lele,,0,0,1,0,346.360,59.116,18.320,4.0450,"Sayur Kukus,Tim Lele,Sayur Kukus,Perkedel Lele,"


In [ ]:
def filter_eer(eer):
  rekomendasi_bersih = rekomendasi_nonalergen.copy()
  

In [ ]:
pip install pandas-tfrecords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import tensorflow as tf

#Creating fake data for demonstration
X_train = pd.DataFrame({'feat1':[1,2,3], 
                  'feat2':['one','two','three']})
training_y = pd.DataFrame({'target': [3.4, 11.67, 44444.1]})

X_train.to_csv('X_train.csv')
training_y.to_csv('training_y.csv')

#TFRecords boilerplate
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(index, feat1, feat2, target):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
      'index': _int64_feature(index),
      'feat1': _int64_feature(feat1),
      'feat2': _bytes_feature(feat2),
      'target': _float_feature(target)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

#Loading the data into chunks of size 2.  Change this to 1e5 in your code
CHUNKSIZE = 1e5
train = pd.read_csv('X_train.csv', chunksize=CHUNKSIZE)
y = pd.read_csv('training_y.csv', chunksize=CHUNKSIZE)

file_num = 0
while 1:
    try:
        print(f'{file_num}')
        #Getting the data from the two files 
        df = pd.concat([train.get_chunk(), y.get_chunk()],1)
        
        #Writing the TFRecord
        with tf.io.TFRecordWriter(f'Record_{file_num}.tfrec') as writer:
            for k in range(df.shape[0]):
                row = df.iloc[k,:]
                example = serialize_example(
                    df.index[k],
                    row['feat1'],
                    str.encode(row['feat2']), #Note the str.encode to make tf play nice with strings
                    row['target']) 
                writer.write(example)    
        file_num += 1
    except:
        print(f'ERROR: {sys.exc_info()[0]}')
        break

In [ ]:
from pandas_tfrecords import pd2tf, tf2pd
pd2tf(komposisi_resep_filtered, './tfrecords')
pd2tf(df, './tfrecords')

In [ ]:
komposisi_resep_filtered_tfrc="/content/tfrecords/part-00000-16e21665-adfa-485e-8603-add1c53a7796.tfrecords.gz"
feedback_tfrc = "/content/tfrecords/part-00000-94c16ee6-5976-464c-a53c-8fc813d72bc5.tfrecords.gz"

raw1 = tf.data.TFRecordDataset(komposisi_resep_filtered_tfrc)
for raw_record in raw1.take(2):
    print(repr(raw_record))

DataLossError: ignored